In [1]:
import os
import random
import numpy as np
import tensorflow as tf

# Imposta il seed globale
seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Normalization, Dense, Dropout, ReLU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard, ProgbarLogger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import h5py
import sys
import importlib

# Imposta float32 come tipo predefinito in Keras
tf.keras.backend.set_floatx('float32')

# Tipo da usare esplicitamente (es. in numpy)
force_datatype = np.float32

use_gpu = 1

2025-04-09 16:29:58.397599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744208998.410832   12003 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744208998.414381   12003 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744208998.424805   12003 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744208998.424824   12003 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744208998.424826   12003 computation_placer.cc:177] computation placer alr

In [2]:
# ------------------ Configurazione GPU ------------------ #
if not use_gpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disabilita la GPU
print(f"Using GPU: {tf.config.list_physical_devices('GPU')}")

Using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Load Dataset XTrain

In [3]:
base_folder = '/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/'
input_folder = base_folder + 'Output Matlab/'

#DeepMIMO_dataset_folder = input_folder + 'DeepMIMO Dataset/'
DL_dataset_folder = input_folder + 'DL Dataset/'
network_folder_in = input_folder + 'Neural Network/'

output_folder = base_folder + 'Output Python/'
network_folder_out = output_folder + 'Neural Network/'
figure_folder = output_folder + 'Figures/'

#simulate_data = 1

#if simulate_data == 1:
    # Simulated data for testing purposes
    #dataset_size = 100
    #training_size = 80 
    #validation_size = 20
    #M = 8
#else:
    #dataset_size = 36200
    #training_size = 30000
    #validation_size = 6200
    #M = 64*64 # 4096

#DL_input_reshaped = np.random.rand(M, 1, 1, dataset_size) 
#DL_output_reshaped = np.random.rand(1, 1, M, dataset_size)

My_ar = [32, 64]
Mz_ar = [32, 64]
Mx = 1
My = My_ar[1]
Mz = Mz_ar[1]

M_bar=8
Ur_rows = [1000, 1200]
#              0    1      2      3      4      5      6
Training_Size=[2, 10000, 14000, 18000, 22000, 26000, 30000]
print(Training_Size)
Training_Size_dd = Training_Size[4]
print("Training_Size_dd:", Training_Size_dd)

#Import .mat files of datasets splits
filename_XTrain = DL_dataset_folder + 'XTrain' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'
filename_YTrain = DL_dataset_folder + 'YTrain' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'
filename_XValidation = DL_dataset_folder + 'XValidation' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'
filename_YValidation = DL_dataset_folder + 'YValidation' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'

print(filename_XTrain)
print(filename_YTrain)
print(filename_XValidation)
print(filename_YValidation)

# Load the data using h5py for MATLAB v7.3 files
with h5py.File(filename_XTrain, 'r') as f:
    X_train = np.array(f['XTrain'][:], dtype=force_datatype)
with h5py.File(filename_YTrain, 'r') as f:
    Y_train = np.array(f['YTrain'][:], dtype=force_datatype)
with h5py.File(filename_XValidation, 'r') as f:
    X_val = np.array(f['XValidation'][:], dtype=force_datatype)
with h5py.File(filename_YValidation, 'r') as f:
    Y_val = np.array(f['YValidation'][:], dtype=force_datatype)

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

[2, 10000, 14000, 18000, 22000, 26000, 30000]
Training_Size_dd: 22000
/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output Matlab/DL Dataset/XTrain_seed0_grid1200_M6464_Mbar8_22000.mat
/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output Matlab/DL Dataset/YTrain_seed0_grid1200_M6464_Mbar8_22000.mat
/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output Matlab/DL Dataset/XValidation_seed0_grid1200_M6464_Mbar8_22000.mat
/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output Matlab/DL Dataset/YValidation_seed0_grid1200_M6464_Mbar8_22000.mat
(22000, 1, 1, 1024)
(22000, 4096, 1, 1)
(6200, 1, 1, 1024)
(6200, 4096, 1, 1)


In [4]:
Y_train.dtype

dtype('float32')

## Load Dataset DL_input_reshape

In [5]:
filename_DL_input_reshaped = DL_dataset_folder + 'DL_input_reshaped' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '.mat'
filename_DL_output_reshaped = DL_dataset_folder + 'DL_output_reshaped' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '.mat'
filename_RandP_all = DL_dataset_folder + 'RandP_all' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '.mat'

# Load the data using h5py for MATLAB v7.3 files
with h5py.File(filename_DL_input_reshaped, 'r') as f:
    DL_input_reshaped = np.array(f['DL_input_reshaped'][:], dtype=force_datatype)
with h5py.File(filename_DL_output_reshaped, 'r') as f:
    DL_output_reshaped = np.array(f['DL_output_reshaped'][:], dtype=force_datatype)
with h5py.File(filename_RandP_all, 'r') as f:
    RandP_all = np.array(f['RandP_all'][:], dtype=force_datatype)

print(DL_input_reshaped.shape)
print(DL_output_reshaped.shape)
print(RandP_all.shape)

(36200, 1, 1, 1024)
(36200, 4096, 1, 1)
(1, 36200)


In [6]:
# Flatten the input and output arrays if necessary
#X = DL_input_reshaped.reshape(DL_input_reshaped.shape[0], -1).astype(np.float32)
#Y = DL_output_reshaped.reshape(DL_output_reshaped.shape[0], -1).astype(np.float32)

# Split the dataset into training and validation sets
#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=validation_size / (training_size + validation_size), shuffle=False, random_state=seed)

RandP_all2 = np.squeeze(np.array(RandP_all.astype(int))) - 1

Training_Ind = RandP_all2[0:Training_Size_dd]

Validation_Size = 6200
Validation_Ind = RandP_all2[-Validation_Size:]

print(Training_Ind.shape)
print(Validation_Ind.shape)

X_train = np.array(DL_input_reshaped[Training_Ind, :, :, :], dtype=force_datatype)
Y_train = np.array(DL_output_reshaped[Training_Ind, :, :, :], dtype=force_datatype)
X_val = np.array(DL_input_reshaped[Validation_Ind, :, :, :], dtype=force_datatype)
Y_val = np.array(DL_output_reshaped[Validation_Ind, :, :, :], dtype=force_datatype)

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(22000,)
(6200,)
(22000, 1, 1, 1024)
(22000, 4096, 1, 1)
(6200, 1, 1, 1024)
(6200, 4096, 1, 1)


## Load Matlab trainedNet model

In [7]:
# Costruisci il path e il nome del modulo
modulename = 'trainedNet' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd)
print(modulename)

# Aggiungi la cartella al path (quella che contiene la cartella del modello)
sys.path.append(network_folder_in)
filename_module = os.path.join(network_folder_in, modulename)

# Cambia la working directory temporaneamente
old_cwd = os.getcwd()
os.chdir(network_folder_in)

# Importa il modulo ed esegui il load
myModel = importlib.import_module(modulename)
#import model as myModel
model_mat = myModel.load_model()

# Ripristina la working directory originale
os.chdir(old_cwd)

# Ora puoi usare il modello
model_mat.summary()

print(model_mat.layers[1].mean.shape) # layers[1] is the Normalization layer
print(np.array(model_mat.layers[1].mean))

trainedNet_seed0_grid1200_M6464_Mbar8_22000


I0000 00:00:1744209014.807945   12003 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13709 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:47:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_unnormalized (InputLayer) │ (None, 1024, 1, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ input_ (Normalization)          │ (None, 1024, 1, 1)     │         2,049 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully1_preFlatten1 (Reshape)    │ (None, 1, 1, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully1_ (Dense)                 │ (None, 1, 1, 4096)     │     4,198,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully2_preFlatten1 (Reshape)    │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully2_ (Dense)                 │ (None, 1, 1, 16384)    │    67,125,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 1, 1, 16384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 1, 16384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully3_preFlatten1 (Reshape)    │ (None, 1, 1, 16384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully3_ (Dense)                 │ (None, 1, 1, 16384)    │   268,451,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 1, 1, 16384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1, 1, 16384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully4_preFlatten1 (Reshape)    │ (None, 1, 1, 16384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully4_ (Dense)                 │ (None, 1, 1, 4096)     │    67,112,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 406,890,497 (1.52 GB)

 Trainable params: 406,888,448 (1.52 GB)

 Non-trainable params: 2,049 (8.01 KB)

(1, 1024, 1, 1)
[[[[6.911748e-05]]

  [[6.911748e-05]]

  [[6.911748e-05]]

  ...

  [[6.911748e-05]]

  [[6.911748e-05]]

  [[6.911748e-05]]]]


In [8]:
model_mat.layers[3].dtype

'float32'

## Prediction (!!! Before running, check DL_input_reshap concat comment)

In [9]:
if Training_Size_dd != 30000:
    print("Fig 7 is only for Training_Size_dd = 30000")
else:
    # ------------------ DL Model Prediction ------------------ #
    print("Start DL prediction for Figure 7...")

    # Concatena XTrain + XValidation, cioè utilizza DL_input_reshaped
    #DL_input_reshaped = tf.concat([X_train, X_val], axis=0)
    print(DL_input_reshaped.shape)  # (36200, 1, 1, 1024)

    DL_input_reshaped = tf.transpose(DL_input_reshaped, perm=[0, 3, 1, 2]) 
    print(DL_input_reshaped.shape)  # (36200, 1024, 1, 1)

    # Esegui predizione con DL_input_reshaped
    YPredictedFig7_mat = model_mat.predict(DL_input_reshaped, batch_size=1, verbose=1)
    print(YPredictedFig7_mat.shape)

    # Recupera gli indici dei codebook
    #Indmax_DL_mat = np.argmax(YPredictedFig7_mat, axis=1)
    #print(Indmax_DL_mat.shape) # (36200, 1)

Fig 7 is only for Training_Size_dd = 30000


In [10]:
if Training_Size_dd != 30000:
    print("Fig 7 is only for Training_Size_dd = 30000")
else:
    filename_YPredictedFig7_mat = network_folder_out + 'YPredictedFig7_mat' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd+Validation_Size) + '.mat'

    # Scrittura in formato HDF5 (compatibile MATLAB v7.3)
    with h5py.File(filename_YPredictedFig7_mat, 'w') as f:
        f.create_dataset('YPredictedFig7_mat', data=YPredictedFig7_mat)

    print(f"Variabile salvata in {filename_YPredictedFig7_mat}")

Fig 7 is only for Training_Size_dd = 30000


In [11]:
print("Start DL prediction for Figure 12...")

print(X_val.shape)  # (6200, 1, 1, 1024)

X_val_reshaped = tf.transpose(X_val, perm=[0, 3, 1, 2]) 
print(X_val_reshaped.shape)  # (6200, 1024, 1, 1)

# Esegui predizione con DL_input_reshaped
YPredicted_mat = model_mat.predict(X_val_reshaped, batch_size=1, verbose=1)
print(YPredicted_mat.shape)

Start DL prediction for Figure 12...
(6200, 1, 1, 1024)
(6200, 1024, 1, 1)


/home/work_wsl/miniconda3/envs/deepmimo/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_unnormalized']
Received: inputs=Tensor(shape=(1, 1024, 1, 1))
  warnings.warn(msg)
I0000 00:00:1744209024.943950   12210 service.cc:152] XLA service 0x7f3e24006110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744209024.944003   12210 service.cc:160]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
I0000 00:00:1744209024.982448   12210 cuda_dnn.cc:529] Loaded cuDNN version 90300


  31/6200 ━━━━━━━━━━━━━━━━━━━━ 32s 5ms/step 

I0000 00:00:1744209028.852032   12210 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6200/6200 ━━━━━━━━━━━━━━━━━━━━ 37s 5ms/step
(6200, 4096)


In [12]:
filename_YPredicted_mat = network_folder_out + 'YPredicted_mat' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'

# Scrittura in formato HDF5 (compatibile MATLAB v7.3)
with h5py.File(filename_YPredicted_mat, 'w') as f:
    f.create_dataset('YPredicted_mat', data=YPredicted_mat)

print(f"Variabile salvata in {filename_YPredicted_mat}")

Variabile salvata in /mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output Python/Neural Network/YPredicted_mat_seed0_grid1200_M6464_Mbar8_22000.mat
